In [2]:
# audio_dataset.ipynb 에서 만든 데이터셋 불러오기

import datasets

minds = datasets.load_from_disk("C:/Users/user/angry_level_model/dataset_01")

FileNotFoundError: Directory C:/Users/user/angry_level_model/dataset_01 not found

In [ ]:
label2id = {'분노':'1','중립':'0'}
id2label = {'1':'분노','0':'중립'}
num_labels = len(id2label)